Loading data

In [1]:
import pandas as pd

In [22]:
cell_stats = pd.read_csv('cell_stats.csv.gz')
connections = pd.read_csv('connections.csv.gz')
connections_no_threshold = pd.read_csv('connections_no_threshold.csv.gz')
connectivity_tags = pd.read_csv('connectivity_tags.csv.gz')
consolidated_cell_types = pd.read_csv('consolidated_cell_types.csv.gz')
coordinates = pd.read_csv('coordinates.csv.gz')
names = pd.read_csv('names.csv.gz')
neurons = pd.read_csv('neurons.csv.gz')
neuropil_synapse_table = pd.read_csv('neuropil_synapse_table.csv.gz')
processed_labels = pd.read_csv('processed_labels.csv.gz')

Data preprocessing

In [23]:
cell_stats

,root_id,length_nm,area_nm,size_nm
0,720575940596125868,396840,628299136,37616302080
1,720575940597856265,1184013,1634424448,100004290560
2,720575940597944841,784742,1022307072,58954229760
3,720575940598267657,1645960,2346787840,140744601600
4,720575940599333574,575383,1366784768,111170764800
...,...,...,...,...
139241,720575940661333889,1292208,2567808384,194386442240
139242,720575940661335681,3800832,8201410304,745657835520
139243,720575940661336193,854317,1337822848,90069493760
139244,720575940661337217,948687,1959185664,135310899200


In [24]:
cell_features = cell_stats[['root_id', 'length_nm', 'area_nm', 'size_nm']].copy()
cell_features = cell_features.fillna(0)  # Handle missing values

In [25]:
cell_types = pd.get_dummies(consolidated_cell_types[['root_id', 'primary_type']], columns=['primary_type'])


In [26]:
neuropil_features = neuropil_synapse_table.groupby('root_id').agg({
    'input synapses': 'sum',
    'output synapses': 'sum'
}).reset_index()

In [27]:
neuron_features = neurons[['root_id', 'nt_type', 'nt_type_score']].copy()


In [28]:
connectivity_tags['connectivity_tag'] = connectivity_tags['connectivity_tag'].str.split(',')


In [29]:
node_features = cell_features.merge(cell_types, on='root_id', how='left')\
                             .merge(neuropil_features, on='root_id', how='left')\
                             .merge(neuron_features, on='root_id', how='left')\
                             .merge(connectivity_tags, on='root_id', how='left')\
                             .merge(names, on='root_id', how='left')\
                             .merge(processed_labels, on='root_id', how='left')
node_features = node_features.fillna(0)

normalize features

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
node_features[['length_nm', 'area_nm', 'size_nm']] = scaler.fit_transform(node_features[['length_nm', 'area_nm', 'size_nm']])

In [34]:
node_features.to_csv('preprocessed_node_features.csv', index=False)